In [275]:
import os
import json
import pandas as pd
from datetime import datetime, timedelta
from nba_api.live.nba.endpoints import scoreboard
import warnings

# Today's Score Board
games = scoreboard.ScoreBoard()

# Get the dictionary representation of the data
games_data = games.get_dict()

# Extract the 'games' key from the dictionary
games_list = games_data['scoreboard']['games']

# Convert the list of games to a DataFrame
games_df = pd.DataFrame(games_list)

games_df

,gameId,gameCode,gameStatus,gameStatusText,period,gameClock,gameTimeUTC,gameEt,regulationPeriods,ifNecessary,seriesGameNumber,seriesText,seriesConference,poRoundDesc,gameSubtype,homeTeam,awayTeam,gameLeaders,pbOdds
0,0022300244,20231125/DALLAC,2,Q4 5:20,4,PT05M20.00S,2023-11-26T03:30:00Z,2023-11-25T22:30:00-05:00,4,False,,,,,,"{'teamId': 1610612746, 'teamName': 'Clippers',...","{'teamId': 1610612742, 'teamName': 'Mavericks'...","{'homeLeaders': {'personId': 202331, 'name': '...","{'team': None, 'odds': 0.0, 'suspended': 0}"
1,0022300239,20231125/PHIOKC,3,Final,4,,2023-11-25T22:00:00Z,2023-11-25T17:00:00Z,4,False,,,,,,"{'teamId': 1610612760, 'teamName': 'Thunder', ...","{'teamId': 1610612755, 'teamName': '76ers', 't...","{'homeLeaders': {'personId': 1628983, 'name': ...","{'team': None, 'odds': 0.0, 'suspended': 0}"
2,0022300240,20231125/MIABKN,3,Final,4,,2023-11-25T23:00:00Z,2023-11-25T18:00:00Z,4,False,,,,,,"{'teamId': 1610612751, 'teamName': 'Nets', 'te...","{'teamId': 1610612748, 'teamName': 'Heat', 'te...","{'homeLeaders': {'personId': 1629661, 'name': ...","{'team': None, 'odds': 0.0, 'suspended': 0}"
3,0022300241,20231125/ATLWAS,3,Final,4,,2023-11-26T00:00:00Z,2023-11-25T19:00:00Z,4,False,,,,,,"{'teamId': 1610612764, 'teamName': 'Wizards', ...","{'teamId': 1610612737, 'teamName': 'Hawks', 't...","{'homeLeaders': {'personId': 1628398, 'name': ...","{'team': None, 'odds': 0.0, 'suspended': 0}"
4,0022300242,20231125/LALCLE,3,Final,4,,2023-11-26T00:30:00Z,2023-11-25T19:30:00Z,4,False,,,,,,"{'teamId': 1610612739, 'teamName': 'Cavaliers'...","{'teamId': 1610612747, 'teamName': 'Lakers', '...","{'homeLeaders': {'personId': 1628386, 'name': ...","{'team': None, 'odds': 0.0, 'suspended': 0}"
5,0022300243,20231125/NOPUTA,3,Final,4,,2023-11-26T02:30:00Z,2023-11-25T21:30:00Z,4,False,,,,,,"{'teamId': 1610612762, 'teamName': 'Jazz', 'te...","{'teamId': 1610612740, 'teamName': 'Pelicans',...","{'homeLeaders': {'personId': 1641718, 'name': ...","{'team': None, 'odds': 0.0, 'suspended': 0}"


In [276]:
#Clean dataframe

# Columns to drop
columns_to_drop = ['gameCode','gameStatusText','gameId','gameStatus', 'period', 'gameClock', 'gameEt', 'regulationPeriods', 'ifNecessary', 'seriesGameNumber', 'seriesText', 'seriesConference', 'poRoundDesc', 'gameSubtype', 'gameLeaders', 'pbOdds']

# Drop specified columns and any columns with null values
games_df = games_df.drop(columns=columns_to_drop).dropna(axis=1, how='all')

# Extract 'teamName' from 'homeTeam' and 'awayTeam'
games_df['homeTeam'] = games_df['homeTeam'].apply(lambda x: x['teamName'] if isinstance(x, dict) and 'teamName' in x else None)
games_df['awayTeam'] = games_df['awayTeam'].apply(lambda x: x['teamName'] if isinstance(x, dict) and 'teamName' in x else None)

games_df

,gameTimeUTC,homeTeam,awayTeam
0,2023-11-26T03:30:00Z,Clippers,Mavericks
1,2023-11-25T22:00:00Z,Thunder,76ers
2,2023-11-25T23:00:00Z,Nets,Heat
3,2023-11-26T00:00:00Z,Wizards,Hawks
4,2023-11-26T00:30:00Z,Cavaliers,Lakers
5,2023-11-26T02:30:00Z,Jazz,Pelicans


In [277]:
# Read abbreviations from the JSON file into a dictionary
json_path = os.path.join('datasets', 'teams.json')
with open(json_path, 'r') as file:
    team_name_to_abbr = json.load(file)

# Function to replace team names with abbreviations
def replace_team_with_abbr(team_name):
    for abbr, full_name in team_name_to_abbr.items():
        if team_name in full_name:
            return abbr
    return team_name

# Replace team names with abbreviations in the homeTeam and awayTeam columns
games_df['homeTeam'] = games_df['homeTeam'].apply(replace_team_with_abbr)
games_df['awayTeam'] = games_df['awayTeam'].apply(replace_team_with_abbr)

games_df


,gameTimeUTC,homeTeam,awayTeam
0,2023-11-26T03:30:00Z,LAC,DAL
1,2023-11-25T22:00:00Z,OKC,PHI
2,2023-11-25T23:00:00Z,BKN,MIA
3,2023-11-26T00:00:00Z,WAS,ATL
4,2023-11-26T00:30:00Z,CLE,LAL
5,2023-11-26T02:30:00Z,UTA,NOP


In [278]:
# Split 'gameTimeUTC' into 'date' and 'time' manually
games_df['date'] = games_df['gameTimeUTC'].str.split('T').str[0]
games_df['time'] = games_df['gameTimeUTC'].str.split('T').str[1].str[:-1]  # Remove the 'Z' at the end

# Convert 'time' column to PST in AM/PM format
games_df['time'] = pd.to_datetime(games_df['time'], format='%H:%M:%S') - pd.Timedelta(hours=8)
games_df['time'] = games_df['time'].dt.strftime('%I:%M %p').str.lstrip('0')  # Remove leading zero

# Drop the 'gameTimeUTC' column
games_df = games_df.drop('gameTimeUTC', axis=1)

games_df


,homeTeam,awayTeam,date,time
0,LAC,DAL,2023-11-26,7:30 PM
1,OKC,PHI,2023-11-25,2:00 PM
2,BKN,MIA,2023-11-25,3:00 PM
3,WAS,ATL,2023-11-26,4:00 PM
4,CLE,LAL,2023-11-26,4:30 PM
5,UTA,NOP,2023-11-26,6:30 PM


In [279]:
# Set the relative path to the 'datasets' folder
relative_path = os.path.join(os.getcwd(), 'datasets')

# Save the DataFrame to a CSV file in the 'datasets' folder
games_df.to_csv(os.path.join(relative_path, 'games.csv'), index=False)

games_df

,homeTeam,awayTeam,date,time
0,LAC,DAL,2023-11-26,7:30 PM
1,OKC,PHI,2023-11-25,2:00 PM
2,BKN,MIA,2023-11-25,3:00 PM
3,WAS,ATL,2023-11-26,4:00 PM
4,CLE,LAL,2023-11-26,4:30 PM
5,UTA,NOP,2023-11-26,6:30 PM


In [280]:
# Specify the relative paths to the CSV files
games_path = 'datasets/games.csv'
predict_data_path = 'datasets/predict.csv'
processed_data_path = 'datasets/processed_data.csv'

read_file = os.path.join(os.getcwd(), processed_data_path)
# Read the processed data CSV file to get column headers
processed_data_df = pd.read_csv(read_file, index_col = 0)
column_headers = processed_data_df.columns.tolist()

# Create an empty DataFrame with the column headers
predict_df = pd.DataFrame(columns=column_headers)

# Read the games.csv file
games_df = pd.read_csv(games_path)

# # Ignore FutureWarnings related to setting items of incompatible dtype
# warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")

# Iterate through each game in games_df
for index, row in games_df.iterrows():
    # Create a row for the home team
    home_row = pd.Series(index=column_headers)
    home_row['team_x'] = row['homeTeam']
    home_row['team_opp_next_x'] = row['awayTeam']
    home_row['team_y'] = row['awayTeam']
    home_row['team_opp_next_y'] = row['homeTeam']
    home_row['home_next'] = 1
    home_row['date_next'] = row['date']
    
    # Use df.loc to append the row to predict_df
    predict_df.loc[len(predict_df)] = home_row

    # Create a row for the away team
    away_row = pd.Series(index=column_headers)
    away_row['team_x'] = row['awayTeam']
    away_row['team_opp_next_x'] = row['homeTeam']
    away_row['home_next'] = 0
    away_row['team_y'] = row['homeTeam']
    away_row['team_opp_next_y'] = row['awayTeam']
    away_row['date_next'] = row['date']

    # Use df.loc to append the row to predict_df
    predict_df.loc[len(predict_df)] = away_row
    
    # Select columns containing 'season'
    season_columns = predict_df.filter(like='season')

    # Set all cells in the selected columns to 2023
    predict_df.loc[len(predict_df) - 1, season_columns.columns] = 2023
    predict_df.loc[len(predict_df) - 2, season_columns.columns] = 2023

# Display the resulting DataFrame
predict_df


/var/folders/10/zgb5222d30nfb8w2mswkfr000000gn/T/ipykernel_95453/1196498827.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'LAC' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  home_row['team_x'] = row['homeTeam']
/var/folders/10/zgb5222d30nfb8w2mswkfr000000gn/T/ipykernel_95453/1196498827.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'DAL' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  away_row['team_x'] = row['awayTeam']
/var/folders/10/zgb5222d30nfb8w2mswkfr000000gn/T/ipykernel_95453/1196498827.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'OKC' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  home_row['team_x'] = row['homeTeam'

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,...,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,team_10_y,season_10_y,team_opp_next_y,team_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,LAC,DAL
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,DAL,LAC
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,OKC,PHI
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,PHI,OKC
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,BKN,MIA
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,MIA,BKN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,WAS,ATL
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,ATL,WAS
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,CLE,LAL
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,LAL,CLE


In [281]:
processed_data_df

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,...,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,team_10_y,season_10_y,team_opp_next_y,team_y
mp,,,,,,,,,,,,,,,,,,,,,
0.00,0.477273,0.500000,0.375598,0.379310,0.348485,0.483373,0.441860,0.396825,0.730455,0.535714,...,0.273427,0.270616,0.478824,0.308654,0.6,1,TOR,2016,SAC,TOR
0.00,0.340909,0.250000,0.413876,0.310345,0.257576,0.509501,0.511628,0.412698,0.827305,0.464286,...,0.124904,0.404739,0.408235,0.428846,0.2,1,SAC,2016,TOR,SAC
0.50,0.409091,0.455882,0.330144,0.482759,0.515152,0.437055,0.372093,0.412698,0.568261,0.500000,...,0.153273,0.344076,0.384706,0.319231,0.7,0,DET,2016,CLE,DET
0.25,0.545455,0.544118,0.416268,0.413793,0.454545,0.419240,0.186047,0.142857,0.883314,0.392857,...,0.276508,0.352607,0.482353,0.316346,0.7,0,TOR,2016,GSW,TOR
0.00,0.340909,0.558824,0.186603,0.206897,0.469697,0.203088,0.139535,0.111111,0.854142,0.321429,...,0.156739,0.470142,0.391765,0.436538,0.6,0,NOP,2016,DEN,NOP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.00,0.545455,0.426471,0.511962,0.448276,0.469697,0.440618,0.372093,0.365079,0.659277,0.535714,...,0.235173,0.562085,0.552941,0.429808,0.4,0,GSW,2022,BOS,GSW
0.00,0.477273,0.455882,0.409091,0.517241,0.590909,0.414489,0.255814,0.222222,0.766628,0.571429,...,0.170603,0.431754,0.522353,0.348077,0.5,0,BOS,2022,GSW,BOS
0.00,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,0.392857,...,0.242875,0.567773,0.575294,0.394231,0.4,1,GSW,2022,BOS,GSW


In [282]:
selected_columns = ['team_x', 'team_opp_next_x','team_y', 'team_opp_next_y', 'home_next', 'date_next', 'season']
predict_df[selected_columns]

,team_x,team_opp_next_x,team_y,team_opp_next_y,home_next,date_next,season
0,LAC,DAL,DAL,LAC,1,2023-11-26,2023
1,DAL,LAC,LAC,DAL,0,2023-11-26,2023
2,OKC,PHI,PHI,OKC,1,2023-11-25,2023
3,PHI,OKC,OKC,PHI,0,2023-11-25,2023
4,BKN,MIA,MIA,BKN,1,2023-11-25,2023
5,MIA,BKN,BKN,MIA,0,2023-11-25,2023
6,WAS,ATL,ATL,WAS,1,2023-11-26,2023
7,ATL,WAS,WAS,ATL,0,2023-11-26,2023
8,CLE,LAL,LAL,CLE,1,2023-11-26,2023
9,LAL,CLE,CLE,LAL,0,2023-11-26,2023


In [283]:
# Iterate through each row in predict_df
for index, row in predict_df.iterrows():
    # Iterate through processed_data_df starting from the last row for team_x
    found_x = False
    for _, team_x_row in processed_data_df[processed_data_df['team_x'].isin([row['team_x'], 'BNK', 'BRK'])].iloc[::-1].iterrows():
        # Find the columns with '10_x' suffix not including 'opp_10_x'
        columns_to_fill_x = team_x_row.index[team_x_row.index.str.endswith('10_x') & ~team_x_row.index.str.endswith('opp_10_x')]

        # Switch 'BRK' to 'BNK'
        if team_x_row['team_x'] == 'BRK':
            team_x_row['team_x'] = 'BNK'

        # Fill in the corresponding columns in predict_df for team_x
        for col_x in columns_to_fill_x:
            predict_df.at[index, col_x] = team_x_row[col_x]
        found_x = True
        break

    # If no matching row is found for team_x, you can handle this case accordingly
    if not found_x:
        print(f"No match found for team_x: {row['team_x']}")

    # Repeat the process for team_y
    found_y = False
    for _, team_y_row in processed_data_df[processed_data_df['team_y'].isin([row['team_y'], 'BNK', 'BRK'])].iloc[::-1].iterrows():
        # Find the columns with '10_y' suffix not including 'opp_10_y'
        columns_to_fill_y = team_y_row.index[team_y_row.index.str.endswith('10_y') & ~team_y_row.index.str.endswith('opp_10_y')]

        # Switch 'BRK' to 'BNK'
        if team_y_row['team_y'] == 'BRK':
            team_y_row['team_y'] = 'BNK'

        # Fill in the corresponding columns in predict_df for team_y
        for col_y in columns_to_fill_y:
            predict_df.at[index, col_y] = team_y_row[col_y]
        found_y = True
        break

    # If no matching row is found for team_y, you can handle this case accordingly
    if not found_y:
        print(f"No match found for team_y: {row['team_y']}")
        
        



In [284]:
# Drop certain columns from predict_df
columns_to_drop = ['season_10_x', 'season_10_y', 'team_10_x', 'team_10_y', 'won_10_y']
predict_df = predict_df.drop(columns=columns_to_drop, errors='ignore')

# Get all columns with '10_x' suffix
selected_columns += predict_df.columns[predict_df.columns.str.endswith('10_x') & ~predict_df.columns.str.endswith('opp_10_x')].tolist()

# Get all columns with '10_y' suffix
selected_columns += predict_df.columns[predict_df.columns.str.endswith('10_y') & ~predict_df.columns.str.endswith('opp_10_y')].tolist()

# Drop columns with NaN values
predict_df = predict_df.dropna(axis=1, how='all')

# Display the selected columns
predict_df[selected_columns]

,team_x,team_opp_next_x,team_y,team_opp_next_y,home_next,date_next,season,mp_10_x,fg_10_x,fga_10_x,...,trb%_max_10_y,ast%_max_10_y,stl%_max_10_y,blk%_max_10_y,tov%_max_10_y,usg%_max_10_y,ortg_max_10_y,drtg_max_10_y,total_10_y,home_10_y
0,LAC,DAL,DAL,LAC,1,2023-11-26,2023,0.025,0.529545,0.386765,...,0.247855,0.438303,0.0842,0.0946,0.370021,0.282157,0.593839,0.552941,0.405769,0.5
1,DAL,LAC,LAC,DAL,0,2023-11-26,2023,0.0,0.388636,0.3,...,0.181848,0.229817,0.0495,0.1045,0.445493,0.137869,0.404265,0.550588,0.493269,0.6
2,OKC,PHI,PHI,OKC,1,2023-11-25,2023,0.025,0.529545,0.386765,...,0.235204,0.388303,0.0492,0.1526,0.410692,0.254942,0.653555,0.516471,0.377885,0.4
3,PHI,OKC,OKC,PHI,0,2023-11-25,2023,0.025,0.381818,0.247059,...,0.181848,0.229817,0.0495,0.1045,0.445493,0.137869,0.404265,0.550588,0.493269,0.6
4,BKN,MIA,MIA,BKN,1,2023-11-25,2023,0.025,0.529545,0.386765,...,0.19736,0.236239,0.0655,0.0849,0.361111,0.146727,0.384834,0.507059,0.353846,0.4
5,MIA,BKN,BKN,MIA,0,2023-11-25,2023,0.0,0.404545,0.367647,...,0.181848,0.229817,0.0495,0.1045,0.445493,0.137869,0.404265,0.550588,0.493269,0.6
6,WAS,ATL,ATL,WAS,1,2023-11-26,2023,0.025,0.529545,0.386765,...,0.234323,0.464679,0.0575,0.0971,0.319182,0.348267,0.473934,0.537647,0.439423,0.4
7,ATL,WAS,WAS,ATL,0,2023-11-26,2023,0.0,0.481818,0.370588,...,0.181848,0.229817,0.0495,0.1045,0.445493,0.137869,0.404265,0.550588,0.493269,0.6
8,CLE,LAL,LAL,CLE,1,2023-11-26,2023,0.025,0.529545,0.386765,...,0.181848,0.229817,0.0495,0.1045,0.445493,0.137869,0.404265,0.550588,0.493269,0.6
9,LAL,CLE,CLE,LAL,0,2023-11-26,2023,0.025,0.529545,0.386765,...,0.181848,0.229817,0.0495,0.1045,0.445493,0.137869,0.404265,0.550588,0.493269,0.6


In [285]:
predict_df[['team_x', 'team_opp_next_x', 'team_y', 'team_opp_next_y']]

,team_x,team_opp_next_x,team_y,team_opp_next_y
0,LAC,DAL,DAL,LAC
1,DAL,LAC,LAC,DAL
2,OKC,PHI,PHI,OKC
3,PHI,OKC,OKC,PHI
4,BKN,MIA,MIA,BKN
5,MIA,BKN,BKN,MIA
6,WAS,ATL,ATL,WAS
7,ATL,WAS,WAS,ATL
8,CLE,LAL,LAL,CLE
9,LAL,CLE,CLE,LAL


In [286]:
# Save the resulting DataFrame to a new CSV file
predict_df.to_csv(predict_data_path, index=False)